In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
def calc_price(phi1, phi2, cost, wtp):
    """compute each insurer's price (assumes interior solution)"""
    phi1, phi2 = phi1, phi2
    p1 = (2*phi1+phi2)/3. + cost
    p2 = (2*phi2+phi1)/3. + cost
    if phi1 <= 0 or phi2 <= 0:
        return 0,0
    
    return p1, p2
  
    
def calc_s(phi1, phi2, cost, wtp):
    """compute the proportion of people choosing each insurer
    assumes interior solution"""
    p1, p2 = calc_price(phi1, phi2, cost, wtp)
    s_hat = .5 + (p2 - p1)/(2.*cost)
    return s_hat


def calc_profits(phi1,phi2,cost, wtp, mc):
    phi1, phi2 = phi1, phi2
    s= calc_s(phi1, phi2, cost, wtp)
    p1,p2 = calc_price(phi1, phi2, cost, wtp)
    profits1, profits2 = s*(p1-phi1), (1-s)*(p2-phi2)
    mc1,mc2 = mc
    hosp_profit = s*(phi1-mc1) +  (1-s)*(phi2-mc2)
    
    if phi1 <= 0 or phi2 <= 0:
        return 0,0,0
    
    return hosp_profit, profits1, profits2

    
def outside_simlt(phi1, phi2, cost, wtp,  mc, active=False):
    """outside option in simult case
    assuming listprice high enough"""
    mc1,mc2 = mc
    
    if active:
        return (wtp-cost)/2 - mc2 #outside cost is other insurer?

    
    s_hat = calc_s(phi1, phi2, cost, wtp) # s_hat with no recapture
    p1,p2 = calc_price(phi1, phi2, cost, wtp)
    s_hat  = np.clip((cost + p2 - wtp)/cost,0,1) #s_hat with recapture
    
    
    return (1-s_hat)*(phi2)


#arbitrary outside option...
def nash_in_nash(phi1, phi2, cost, wtp, mc, beta=.5,outside=0):
    hosp_profit, profits1, profits2 = calc_profits(phi1, phi2,  cost,  wtp, mc)
    obj = -1*(np.log(max(hosp_profit-outside,1e-4))*(1-beta) 
              + np.log(profits1)*beta)
    return obj


COST = 5
WTP = 25
MC = np.array([0,0])

print(nash_in_nash(3*COST,3*COST, COST, WTP, MC))

-1.8121704664881826


In [3]:
def bargain_helper(phi1, phi2, cost, wtp, mc, beta=.5,outside=0):
    """solve each firm 1s optimization holding phi 2 fixed"""
    result = minimize(nash_in_nash, phi1, args=(phi2,cost, wtp, mc, beta,outside),
                      method='Nelder-Mead', options={'disp': False})
    return result.x

COST = 5
WTP  = 25
MC = np.array([0,0])

print(bargain_helper(15,15, COST,WTP,MC,beta=.5,outside=0))

[8.35501099]


In [4]:
def seq_bargain(phi1, cost, wtp, mc, betas=[.5,.5]):
    """solve the bargaining problems seperately and iterate for a solution"""       
    
    #loop variables, check on this...
    phi2 = phi1 +10
    
    diff =  np.maximum(phi1,phi2)
    phi10,phi20 = phi1+.5,phi2+.5
    maxiter = 20
    while maxiter >=0 and diff > 10e-7:
        outside = outside_simlt(phi10, phi20, cost, wtp,  mc, active=True)

        #seems as though there is a contraction mapping here, need to think more about why
        phi2 = bargain_helper(phi20, phi10, cost, wtp, mc[::-1], beta=betas[1],outside=phi10)
        phi1 = bargain_helper(phi10, phi20, cost, wtp, mc, beta=betas[0] ,outside=outside)
        
        #update loop variables
        diff = np.abs(np.maximum(phi1 - phi10,phi2-phi20))[0]
        phi10,phi20 = phi1,phi2
        maxiter = maxiter-1
        
    return phi1, phi2


COST = 5
WTP  = 25
MC = np.array([0,0])
print(seq_bargain(15,COST,WTP,MC))
#print(simult_bargain(15,15,COST,WTP,MC,active=True))

(array([15.62577103]), array([19.37502828]))


In [5]:
#willingness to pay

COST = 5
WTP = 25

print("l" , "phi1", "phi 2", "hosp profit", "profit 1", "profit 2")
for i in range(5):
    mc_i = np.array([0, 0])
    wtp_i = WTP + i
    phi1,phi2 = seq_bargain(20,COST,wtp_i,mc_i)
    hosp_profit, profits1, profits2= calc_profits(phi1[0],phi2[0],COST, wtp_i, mc_i )
    p1,p2=calc_price(phi1, phi2, COST, wtp_i)
    s1=calc_s(phi1, phi2, COST, wtp_i)
    print(wtp_i,wtp_i, round(phi1[0],3), round(phi2[0],3), round(p1[0],3), round(p2[0],3), 
          round(hosp_profit,3), round(profits1,3), round(profits2,3))

l phi1 phi 2 hosp profit profit 1 profit 2
25 25 15.627 19.376 21.876 23.126 17.033 3.906 1.406
26 26 16.127 19.876 22.376 23.626 17.533 3.906 1.406
27 27 16.626 20.376 22.876 24.126 18.033 3.906 1.406
28 28 17.126 20.876 23.376 24.626 18.532 3.906 1.406
29 29 17.626 21.376 23.876 25.126 19.032 3.906 1.406


In [ ]:
#willingness to pay

COST = 5
WTP = 25

print("l" , "phi1", "phi 2", "hosp profit", "profit 1", "profit 2")
for i in range(5):
    mc_i = np.array([0, 0])
    wtp_i = WTP + i
    phi1,phi2 = seq_bargain(20,COST,wtp_i,mc_i)
    hosp_profit, profits1, profits2= calc_profits(phi1[0],phi2[0],COST, wtp_i, mc_i )
    p1,p2=calc_price(phi1, phi2, COST, wtp_i)
    s1=calc_s(phi1, phi2, COST, wtp_i)
    print(wtp_i,wtp_i, round(phi1[0],3), round(phi2[0],3), round(p1[0],3), round(p2[0],3), 
          round(hosp_profit,3), round(profits1,3), round(profits2,3))

In [6]:
#firm 1 high cost
COST = 5
WTP = 25

print("l" , "phi1", "phi 2", "hosp profit", "profit 1", "profit 2")
for i in np.linspace(0,1,3):
    mc_i = np.array([i, 0])
    wtp_i = WTP 
    phi1,phi2 = seq_bargain(20,COST,wtp_i,mc_i)
    p1,p2 = calc_price(phi1, phi2, COST,wtp_i)
    hosp_profit, profits1, profits2= calc_profits(phi1[0],phi2[0],COST, wtp_i, mc_i )
    s1=calc_s(phi1, phi2, COST, wtp_i)
    print(mc_i[0],mc_i[1], round(phi1[0],3), round(phi2[0],3), round(p1[0],3), round(p2[0],3), 
          round(hosp_profit,3), round(profits1,3), round(profits2,3))

l phi1 phi 2 hosp profit profit 1 profit 2
0.0 0.0 15.627 19.376 21.876 23.126 17.033 3.906 1.406
0.5 0.0 16.343 20.403 22.696 24.05 17.506 4.036 1.33
1.0 0.0 17.129 21.546 23.601 25.073 18.04 4.189 1.245


In [7]:
#firm 2 high cost
COST = 5
WTP = 25

print("l" , "phi1", "phi 2", "hosp profit", "profit 1", "profit 2")
for i in np.linspace(0,1,3):
    mc_i = np.array([0, i])
    wtp_i = WTP 
    phi1,phi2 = seq_bargain(20,COST,wtp_i,mc_i)
    p1,p2 = calc_price(phi1, phi2, COST,wtp_i)
    hosp_profit, profits1, profits2= calc_profits(phi1[0],phi2[0],COST, wtp_i, mc_i )
    s1=calc_s(phi1, phi2, COST, wtp_i)
    print(mc_i[0],mc_i[1], round(phi1[0],3), round(phi2[0],3), round(p1[0],3), round(p2[0],3), 
          round(hosp_profit,3), round(profits1,3), round(profits2,3))

l phi1 phi 2 hosp profit profit 1 profit 2
0.0 0.0 15.627 19.376 21.876 23.126 17.033 3.906 1.406
0.0 0.5 15.439 19.564 21.814 23.189 16.753 4.064 1.314
0.0 1.0 15.252 19.752 21.752 23.252 16.477 4.225 1.225


In [ ]:
#generate a bunch of runs with passive and compute costs

def compute_cov(nobs=1000):
    mcs = np.random.uniform(low=0.0, high=2.0, size=(2,nobs))
    phis = []
    for i in range(nobs):
        phi = seq_bargain(30,COST,WTP,mcs[:,i]) 
        phis.append(np.array(phi).flatten())

    phis = np.array(phis)
    return np.cov(phis,rowvar=False),phis

matrix, phis = compute_cov()
print(phis[:,0].max(),phis[:,0].mean(),phis[:,0].min())
print('--')
print(phis[:,1].max(),phis[:,1].mean(),phis[:,1].min())
print('--')
print(matrix)
print('--')
print(np.cov(phis[0:500,:],rowvar=False))
print('--')
print(np.cov(phis[100:600,:],rowvar=False))
print('--')
print(np.cov(phis[200:700,:],rowvar=False))
print('--')
print(np.cov(phis[300:800,:],rowvar=False))


In [ ]:
phis_clean = phis[phis[:,0]>=17]
print('--')
print(np.cov(phis_clean[0:500,:],rowvar=False))
print('--')
print(np.cov(phis_clean[100:600,:],rowvar=False))
print('--')
print(np.cov(phis_clean[200:700,:],rowvar=False))
print('--')
print(np.cov(phis_clean[300:800,:],rowvar=False))

In [ ]:
#generate a bunch of runs with passive and compute costs

def compute_cov(nobs=1000):
    mcs = np.random.uniform(low=0.0, high=1.0, size=(2,nobs))
    phis = []
    for i in range(nobs):
        phi = seq_bargain(30,COST,WTP,mcs[:,i]) 
        phis.append(np.array(phi).flatten())

    phis = np.array(phis)
    return np.cov(phis,rowvar=False),phis

matrix, phis = compute_cov()
print(phis[:,0].max(),phis[:,0].mean(),phis[:,0].min())
print('--')
print(phis[:,1].max(),phis[:,1].mean(),phis[:,1].min())
print('--')
print(matrix)
print('--')
print(np.cov(phis[0:500,:],rowvar=False))
print('--')
print(np.cov(phis[100:600,:],rowvar=False))
print('--')
print(np.cov(phis[200:700,:],rowvar=False))

In [ ]:
#generate a bunch of runs with passive and compute costs

def compute_cov(nobs=1000):
    mcs = np.random.uniform(low=0.0, high=.5, size=(2,nobs))
    phis = []
    for i in range(nobs):
        phi = seq_bargain(30,COST,WTP,mcs[:,i]) 
        phis.append(np.array(phi).flatten())

    phis = np.array(phis)
    return np.cov(phis,rowvar=False),phis

matrix, phis = compute_cov()
print(phis[:,0].max(),phis[:,0].mean(),phis[:,0].min())
print('--')
print(phis[:,1].max(),phis[:,1].mean(),phis[:,1].min())
print('--')
print(matrix)
print('--')
print(np.cov(phis[0:500,:],rowvar=False))
print('--')
print(np.cov(phis[100:600,:],rowvar=False))
print('--')
print(np.cov(phis[200:700,:],rowvar=False))

In [ ]:
#just make sure it works for stuff i know is right

def compute_solution(nobs=20):
    #mcs = np.random.uniform(low=0.0, high=2.0, size=(2,nobs))
    phis = []
    vs = np.random.uniform(low=26,high=26.5,size=(nobs))
    lambdas = np.random.uniform(low=5,high=6,size=(nobs))
    x = np.concatenate(([vs],[lambdas])) #,mcs
    for i in range(nobs):
        phi = seq_bargain(30,lambdas[i],vs[i],np.array([0,0])) 
        phis.append(np.array(phi).flatten())
    phis = np.array(phis)
    xx = x.dot(x.transpose())
    xy1 = x.dot(phis[:,0])
    xy2 = x.dot(phis[:,1])
    return np.linalg.inv(xx).dot(xy1),np.linalg.inv(xx).dot(xy2)
    
print(compute_solution(nobs=10))

In [ ]:
def compute_solution(nobs=20):
    
    #set up x variables to solve the model
    phis = []
    vs = np.random.uniform(low=26,high=26.5,size=(nobs))
    lambdas = np.random.uniform(low=5,high=6,size=(nobs))
    mcs = np.random.uniform(low=0.0, high=1.0, size=(2,nobs))
    x = np.concatenate(([vs],[lambdas],mcs))
    
    #generate a few solutions...
    for i in range(nobs):
        phi = seq_bargain(35,lambdas[i],vs[i],mcs[:,i])
        phis.append(np.array(phi).flatten())
        
    #do regression stuff to solve
    phis = np.array(phis)
    xx = x.dot(x.transpose())
    xy1 = x.dot(phis[:,0])
    xy2 = x.dot(phis[:,1])
    return np.linalg.inv(xx).dot(xy1),np.linalg.inv(xx).dot(xy2)

#coeffs are v, lambda, c1, c2
print(compute_solution(nobs=10))  

#maybe do it 10 times and take the average?